In [1]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from rasterio.merge import merge
import glob

from rasterio.plot import adjust_band
from rasterio.plot import show
from rasterio.plot import reshape_as_raster, reshape_as_image
import rasterio.features
import rasterio.warp
import rasterio.mask
from rasterio.merge import merge

from tensorflow import keras

from skimage.transform import resize

C:\Users\Gavin\anaconda3\envs\Phd_Backup\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Gavin\anaconda3\envs\Phd_Backup\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Gavin\anaconda3\envs\Phd_Backup\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Gavin\anaconda3\envs\Phd_Backup\lib\site-packages

In [2]:
#ensure size of tile is uniform, and reshape the numpy into a image format then normalize
def newXpreprocessing(tile,layers,width,height):
    #print(tile.shape,"before")
    tile = resize(tile, (layers,width,height))
    #print(tile.shape,"resize")
    tile = reshape_as_image(tile)
    #print(tile.shape,"reshape")
    #normalising tile
    tile = tile/255
    return tile

In [3]:
#Keras compatible preditor - works tif. Each image is predicted against and then saved as a colour representing the prediction
# to be turned into a mosaic at the next step
def predictingNewX(model,data_dir_list,data_path,width=40,height=40,layers=3):
    #load list of image names in each category
    img_list = os.listdir(os.path.join(data_path))
     #   print ('Loading the images for prediction -'+'{}\n'.format(dataset))
    num_img_files = len(img_list) 

    for i in range(num_img_files):
        img_name = img_list[i]
        img_filename = os.path.join(data_path,img_name)
        #model expects image formatted a certain way
        X_new = []
        with rasterio.open(img_filename) as ds:
            tile=ds.read()
            #preprocess tille according to model expectations
            pre_tile=newXpreprocessing(tile,layers,width,height)

        # Add tile to array
        X_new.append(pre_tile)
        # Make sure they're numpy arrays (as opposed to lists)
        X_new = np.array(X_new)
        #make prediction against X-new array
        yNew = model.predict_classes(X_new)
        #determine colour of class of interest
        class_image = loadClassTile(yNew[0],class_path)
        with rasterio.open(
            str(output_path) +'\\' + str(i) +'_y.tif',
            'w',
            driver='GTiff',
            height=class_image.shape[1],
            width=class_image.shape[2],
            count=class_image.shape[0],
            dtype=class_image.dtype,
            class_image=ds.crs,
            nodata=None,
            transform=ds.transform
            ) as dst:
            dst.write(class_image)
        print('processed: '+ str(i) +'_y.tif')


In [4]:
#Method to replace a detected class with a single colour image representing that class. 
def loadClassTile(yNew,class_path):
    if yNew == 0:
        classlabel = 'yes.tiff'
    elif yNew == 1:
        classlabel = 'no.tiff'
    else:
        classlabel = 'error.tiff'
    
    with rasterio.open(str(class_path) + '\\' + classlabel) as cs:
        class_image=cs.read()
    return class_image
    

In [5]:
# https://gis.stackexchange.com/questions/348925/merging-rasters-with-rasterio-in-blocks-to-avoid-memoryerror
# https://rasterio.readthedocs.io/en/latest/api/rasterio.merge.html
# https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-mosaic.html

# this script is performed in memory and is thus memroy intensive TO DO: create a rasterio-in-blocks implementation
#a folder is created containig the class images(with CRS), all images within this folder are mosaiced together into a single raster 
def merge_raster_in_folder(src_dir, dest_dir):
    """Merge all the raster (TIFF extension) within the given directory"""
    #simple merges outputs from model and merges them
    #src_dir = r"C:\Data\Test_dataset"
    out_fp = dest_dir + r"\prediction_baseline2.tif"
    # create list of raster files in source folder
    src_files_to_mosaic = []
    os.chdir(src_dir)
    for file in glob.glob("*.tif"):
        src = rasterio.open(file)
        src_files_to_mosaic.append(file)

    # create list of raster objects from the list of raster filename
    sources = [rasterio.open(raster) for raster in src_files_to_mosaic]

    # create array representing all source rasters mosaicked together
    mosaic, out_transform = merge(sources)
    #checkl mosaic is created 
    show(mosaic, cmap='terrain')

    out_meta = src.meta.copy()
    out_meta.update({"driver":"GTiff",
                     "height":mosaic.shape[1],
                     "width":mosaic.shape[2],
                     "transform":out_transform,
                     "crs":'+proj=latlong' #TO DO: Pass in CRS
    }
    )
    with rasterio.open(out_fp, "w", **out_meta) as dest:
        dest.write(mosaic)

In [6]:
#declare path directories
data_path = r'C:\Data\Predict'
output_path = r'C:\Data\output3'
class_path = r'C:\Data\BinaryClasses'
result_path = r'C:\Data'
data_dir_list = os.listdir(data_path)

In [11]:
model = keras.models.load_model('bestmodel')
model.summary()
predictingNewX(model,data_dir_list,data_path,width=83,height=83,layers=6)
merge_raster_in_folder(output_path,result_path)

PermissionError: [Errno 13] Unable to open file (unable to open file: name = 'bestmodel', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)